In [2]:
import requests
import pandas as pd
import time
import os
import sys


# Add the 'scripts' folder to the Python path
sys.path.append(os.path.abspath(os.path.join('../scripts')))

from discord_notifier import send_discord_message

# Ensure the directory exists
def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to insert match details into a CSV file
def insert_match_details(match_details, csv_file='../data/processed/match_details.csv'):
    # Check if the file exists
    if os.path.exists(csv_file):
        df_match_details = pd.read_csv(csv_file)
    else:
        # If it doesn't exist, create an empty DataFrame
        df_match_details = pd.DataFrame(columns=[
            'match_id', 'game_creation', 'game_duration', 'game_mode', 'game_type', 'game_version', 
            'map_id', 'participant_id', 'summoner_id', 'puuid', 'champion_id', 'spell1_id', 'spell2_id', 
            'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'kills', 'deaths', 'assists', 
            'total_damage_dealt', 'total_damage_taken', 'gold_earned', 'vision_score', 
            'total_minions_killed', 'team_position', 'win'
        ])

    # Convert the match_details into a DataFrame
    new_data = pd.DataFrame(match_details, columns=df_match_details.columns)

    # Append new data and remove duplicates
    df_match_details = pd.concat([df_match_details, new_data]).drop_duplicates()

    # Save the updated DataFrame back to CSV
    df_match_details.to_csv(csv_file, index=False)
    print(f"Inserted {len(new_data)} match details into {csv_file}.")

# Function to fetch match details from the Riot API
def fetch_match_details(match_id):
    api_key = "RGAPI-66d709f6-56cc-4fb9-b3cf-91573a2072df"
    url = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching match details for {match_id}: {response.status_code}")
        return None

# Function to pull match IDs and insert match details into CSV
def pull_and_insert_match_details():
    # Read match_ids from match_ids.csv
    if os.path.exists('../data/processed/match_ids.csv'):
        df_match_ids = pd.read_csv('../data/processed/match_ids.csv')
    else:
        print("No match_ids.csv found.")
        return

    # If match_details.csv exists, filter out already processed match_ids
    if os.path.exists('../data/processed/match_details.csv'):
        df_match_details = pd.read_csv('../data/processed/match_details.csv')
        match_ids_with_details = df_match_details['match_id'].unique()
        match_ids_to_fetch = df_match_ids[~df_match_ids['match_id'].isin(match_ids_with_details)]['match_id'].tolist()
    else:
        match_ids_to_fetch = df_match_ids['match_id'].tolist()

    # Process in batches
    batch_size = 100
    count = 1
    total = len(match_ids_to_fetch)
    for i in range(0, len(match_ids_to_fetch), batch_size):
        
        match_details_batch = []
        batch = match_ids_to_fetch[i:i + batch_size]
        
        for match_id in batch:
            time.sleep(1.3)
            print(f"fetching values for match: {match_id}: {count}/{total}")
            match_data = fetch_match_details(match_id)
            count += 1
            if match_data:
                # Extract relevant data for each participant and append to match_details_batch
                for participant in match_data['info']['participants']:
                    match_details_batch.append((
                        match_id,
                        match_data['info']['gameCreation'],
                        match_data['info']['gameDuration'],
                        match_data['info']['gameMode'],
                        match_data['info']['gameType'],
                        match_data['info']['gameVersion'],
                        match_data['info']['mapId'],
                        participant.get('participantId'),
                        participant.get('summonerId'),
                        participant.get('puuid'),
                        participant.get('championId'),
                        participant.get('spell1Id'),
                        participant.get('spell2Id'),
                        participant.get('item0'),
                        participant.get('item1'),
                        participant.get('item2'),
                        participant.get('item3'),
                        participant.get('item4'),
                        participant.get('item5'),
                        participant.get('item6'),
                        participant.get('kills'),
                        participant.get('deaths'),
                        participant.get('assists'),
                        participant.get('totalDamageDealt'),
                        participant.get('totalDamageTaken'),
                        participant.get('goldEarned'),
                        participant.get('visionScore'),
                        participant.get('totalMinionsKilled'),
                        participant.get('teamPosition', 'UNKNOWN'),
                        participant.get('win')
                    ))

        # Insert batch into CSV
        insert_match_details(match_details_batch)

if __name__ == "__main__":
    try:
        pull_and_insert_match_details()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        send_discord_message()


fetching values for match: NA1_4994315272: 1/5146
fetching values for match: NA1_4994186546: 2/5146
fetching values for match: NA1_4994181258: 3/5146
fetching values for match: NA1_4994172007: 4/5146
fetching values for match: NA1_4994167776: 5/5146
fetching values for match: NA1_4994142190: 6/5146
fetching values for match: NA1_5062501601: 7/5146
Error fetching match details for NA1_5062501601: 404
fetching values for match: NA1_4934047133: 8/5146
fetching values for match: NA1_4618443057: 9/5146
Error fetching match details for NA1_4618443057: 404
fetching values for match: NA1_4618403573: 10/5146
Error fetching match details for NA1_4618403573: 404
fetching values for match: NA1_4994158381: 11/5146
fetching values for match: NA1_5062495226: 12/5146
Error fetching match details for NA1_5062495226: 404
fetching values for match: NA1_5019249612: 13/5146
fetching values for match: NA1_5062484783: 14/5146
Error fetching match details for NA1_5062484783: 404
fetching values for match: NA1